# Associating Slogfiles with Nodes, Tasks

 - [ ] [Mismatch: Slogfile Tasks](#Mismatch:-Slogfile-Tasks)
 - [ ] [Mismatch: Files](#Mismatch:-Files)
 - [ ] [Mismatch: monikers](#Mismatch:-monikers)
 - [ ] [Mismatch: gentx tasks](#Mismatch:-gentx-tasks)
 - [ ] [Mismatch: Validators](#Mismatch:-Validators)


### Preface: PyData tools

In [1]:
import pandas as pd
dict(pandas=pd.__version__)

{'pandas': '1.3.2'}

## Task Portal Export

Participants use the https://validateagoric.knack.com/ portal to submit their tasks for review.

In [2]:
!ls -l portal-export/submittedtasks.csv

-rw-r--r-- 1 jupyter jupyter 11025281 Aug 28 16:45 portal-export/submittedtasks.csv


In [3]:
tasksub = pd.read_csv('portal-export/submittedtasks.csv',
                      parse_dates=['Last Date Updated'])
tasksub.tail(3)

,Event,Task,Discord ID,Moniker,Developer,Submission Link,Task Type,Points,Status,Verified,TaskBoardID,Last Date Updated
4626,Stress Test,Restart your validator twice during agorictest-15,luizapplima#8073,Papaya Node,Luiza Do Prado Lima,NaN,Network Task,400,Completed,Not accepted,2887,2021-06-28 10:36:00
4627,Stress Test,Restart your validator twice during agorictest-15,CreATivE8536#5039,isavazh,Mokhamed Savazh,Moniker - isavazh<br /><br />agorictest-13 - 2...,Network Task,400,Completed,Not accepted,2835,2021-06-27 08:06:00
4628,Stress Test,Restart your validator twice during agorictest-15,Edouard Lavidalle | Stakin#2231,Stakin,Edouard Lavidalle,NaN,Network Task,400,Completed,Not accepted,2880,2021-06-28 06:41:00


In [4]:
phase45start = '2021-08-15'
tasksub[tasksub['Last Date Updated'] >= phase45start].groupby(['Event', 'Task'])[['TaskBoardID']].count()

TaskBoardID
Event     Task                                                           
Metering  AMM Trade Part 1: Make a trade on the AMM and g...           82
          AMM Trade Part 2: Find the crank number of the ...           70
          Capture and submit a slog file                               96
          Conduct performance analysis                                 50
          Create a contract that charges fees to users                  2
          Create and submit gentx - Metering                          150
          Launch and maintain Agoric's load generator nod...          100
          Maintain uptime during phase!                                89
          Restart your validator at least three times at ...          112
          Start your validator as part of the genesis cer...          125
          Tweet a GIF about about phase 4.5                            89

## Slogfile tasks submitted

In [5]:
taskslog = tasksub[tasksub['Task'] == 'Capture and submit a slog file'].drop(columns=['Event', 'Task', 'Points', 'Task Type']).set_index('TaskBoardID')
print(len(taskslog))
taskslog.tail()

96


,Discord ID,Moniker,Developer,Submission Link,Status,Verified,Last Date Updated
TaskBoardID,,,,,,,
4635,chris-remus#4082,Chainflow,Chris Remus,slogfile available here -<br /><br />https://c...,Completed,In review,2021-08-27 20:49:00
4622,svAbhishek | OmniFlix Network#1613,OmniFlix Network,Sistla Venkata Abhishek,name : 2021-08-27T22:10:31.625Z-svAbhishek | O...,Completed,In review,2021-08-27 18:25:00
4623,Coolex#3021,coolex,Anatolii Shardubin,uploaded via https://submit.agoric.app,Completed,In review,2021-08-27 18:48:00
4641,mr_vavilon#6450,mrvavilon,Egor Malakhov,Completted<br />https://drive.google.com/file/...,Completed,In review,2021-08-28 01:30:00
4645,aliefaisala#3118,aliefaisala,ALIEF FAISAL ANWAR,Submitted.<br />File name : slogfile-upload-5/...,Completed,In review,2021-08-28 12:41:00


## Slogfiles Collected

The https://submit.agoric.app tool authenticated participants using Discord OAuth
and stored their submissions in the `slogfile-upload-5` bucket in Google Cloud Storage.

In [6]:
up5 = !gsutil ls -l gs://slogfile-upload-5/

In [39]:
def pathparts(row):
    filename = row.path.split('/')[-1]
    noext = filename.replace('.slog.gz', '')
    fresh, discordID = noext.split('Z-', 1)
    return dict(filename=filename, fresh=fresh, discordID=discordID)

sf5 = pd.DataFrame.from_records(
    [dict(size=int(size), modified=modified, path=path) for line in up5 if 'Z' in line
     for (size, modified, path) in [line.strip().split('  ', 2)]])
sf5 = pd.concat([sf5, sf5[['path']].apply(pathparts, axis=1, result_type='expand')], axis=1)
sf5 = sf5.set_index('filename').sort_index()
sf5 = sf5.assign(modified=pd.to_datetime(sf5.modified),
                 fresh=pd.to_datetime(sf5.fresh))
print(sf5.dtypes)
sf5.drop(columns=['path']).sort_values('size')

size                       int64
modified     datetime64[ns, UTC]
path                      object
fresh             datetime64[ns]
discordID                 object
dtype: object


,size,modified,fresh,discordID
filename,,,,
2021-08-25T23:59:53.271Z-cw#1047.slog.gz,4921360,2021-08-26 00:07:57+00:00,2021-08-25 23:59:53.271,cw#1047
2021-08-27T10:27:06.530Z-sebytza05#9060.slog.gz,6039130,2021-08-27 10:27:51+00:00,2021-08-27 10:27:06.530,sebytza05#9060
2021-08-26T19:34:10.967Z-Alex Turetskiy#5973.slog.gz,68125044,2021-08-26 19:36:49+00:00,2021-08-26 19:34:10.967,Alex Turetskiy#5973
2021-08-27T07:38:48.952Z-Chococat#1678.slog.gz,88838306,2021-08-27 07:39:39+00:00,2021-08-27 07:38:48.952,Chococat#1678
2021-08-27T16:20:46.088Z-Kirill S#3150.slog.gz,103581352,2021-08-27 16:26:56+00:00,2021-08-27 16:20:46.088,Kirill S#3150
...,...,...,...,...
2021-08-27T16:44:00.475Z-jjangg96#7937.slog.gz,3743327924,2021-08-27 16:55:29+00:00,2021-08-27 16:44:00.475,jjangg96#7937
2021-08-27T12:00:56.385Z-mirxl#0530.slog.gz,3925214742,2021-08-27 13:57:03+00:00,2021-08-27 12:00:56.385,mirxl#0530
2021-08-28T18:13:03.501Z-Kosmos.Infrastructure#7451.slog.gz,5894127057,2021-08-28 18:25:14+00:00,2021-08-28 18:13:03.501,Kosmos.Infrastructure#7451


### Duplicate, empty uploads

These were moved to `dup-test`.

In [8]:
sf5[sf5.duplicated(['discordID', 'size'], keep=False)]

,size,modified,path,fresh,discordID
filename,,,,,


In [9]:
sfold = sf5[sf5.duplicated(['discordID', 'size'], keep='last')]
sfold.path

Series([], Name: path, dtype: object)

In [10]:
from slogdata import CLI

def _cli(bin):
    from subprocess import run, Popen
    return CLI(bin, run, Popen)

gsutil = _cli('gsutil')

In [11]:
# gsutil.run('mv', *sfold.path, 'gs://slogfile-upload-5/dup-test/');

## Connecting Slogfiles with Task Submissions

In [12]:
sf5match = pd.merge(taskslog.reset_index(), sf5.reset_index(),
         left_on='Discord ID', right_on='discordID', how='left'
                   ).set_index('TaskBoardID').drop(columns=['Developer', 'Status', 'path', 'fresh', 'discordID', 'modified'])

sf5match.loc[~sf5match.filename.isnull(), 'Verified'] = 'Accepted'

sf5match = sf5match.sort_values(['Verified', 'Discord ID', 'filename'])

def move_col(df, col, pos):
    x = df[col]
    df = df.drop(columns=[col])
    df.insert(pos, col, x)
    return df

sf5match = move_col(sf5match, 'Verified', 1)
sf5match.head()

,Discord ID,Verified,Moniker,Submission Link,Last Date Updated,filename,size
TaskBoardID,,,,,,,
4566,4andr#6771,Accepted,4andr,provided to you via https://submit.agoric.app<...,2021-08-27 04:52:00,2021-08-27T07:52:31.251Z-4andr#6771.slog.gz,1.720723e+09
4360,Adorid | adorid.xyz#1293,Accepted,Adorid,Uploaded,2021-08-26 09:15:00,2021-08-26T13:07:51.018Z-Adorid | adorid.xyz#1...,1.646891e+09
4227,Alex Turetskiy#5973,Accepted,TuretskiyV,TuretskiyV-agorictest17-chain:<br />https://dr...,2021-08-25 19:30:00,2021-08-26T19:34:10.967Z-Alex Turetskiy#5973.s...,6.812504e+07
4578,Aliaksandr#3647,Accepted,Coolcat,CoolCat<br /><br />https://drive.google.com/fi...,2021-08-27 09:52:00,2021-08-27T14:45:02.111Z-Aliaksandr#3647.slog.gz,7.735847e+08
4499,Anna1242#2262,Accepted,dreamstaker,"I uploaded it on submit.agoric.app, but due to...",2021-08-26 15:34:00,2021-08-26T19:12:39.249Z-Anna1242#2262.slog.gz,1.795795e+09


In [13]:
sf5match.groupby('Verified')[['Discord ID']].aggregate('nunique')

,Discord ID
Verified,
Accepted,54
In review,41


### Mismatch: Slogfile Tasks

with no matching file

In [14]:
sf5match[sf5match.Verified != 'Accepted']

,Discord ID,Verified,Moniker,Submission Link,Last Date Updated,filename,size
TaskBoardID,,,,,,,
4336,#0485,In review,AlphaBravo,Complete,2021-08-26 08:05:00,NaN,NaN
4610,13610152128#5923,In review,nodestradamus,nodestradamus-agorictest17-chain.slog.gz,2021-08-27 15:41:00,NaN,NaN
4629,86b#6313,In review,oraclegen,https://agoric.s3.us-east-2.amazonaws.com/orac...,2021-08-27 19:48:00,NaN,NaN
4389,@calin_stakeborg,In review,Stakeborg,Slog file uploaded on https://submit.agoric.ap...,2021-08-26 10:52:00,NaN,NaN
4504,AndreyRudi#2065,In review,rudolfhe-55,https://storage.nodes.guru/rudolfhe-55-agorict...,2021-08-26 15:51:00,NaN,NaN
4398,ColinkaMalinka#1565,In review,ColinkaMalinka,https://disk.yandex.by/d/PdcjuyhLCYgIFw,2021-08-26 11:12:00,NaN,NaN
4611,CreATivE8536#5039,In review,isavazh,Validator: isavazh<br /><br />https://www.drop...,2021-08-27 15:47:00,NaN,NaN
4418,Danil Ushakov#5735,In review,ushakov,https://drive.google.com/file/d/19QPpwKpB7E999...,2021-08-26 11:37:00,NaN,NaN
4478,EdwardMorra#5755,In review,EdwardMorra,I've uploaded the file to your portal but sinc...,2021-08-26 14:37:00,NaN,NaN


In [15]:
!mkdir -p portal-review

In [16]:
sf5match.to_csv('portal-review/capture_submit_slog.csv')

### Mismatch: Files

with no matching task

In [17]:
sf5[~sf5.discordID.isin(taskslog['Discord ID'])][['discordID']] #.reset_index(drop=True)

,discordID
filename,
2021-08-26T07:02:23.023Z-JoseCT | stakely.io#8451.slog.gz,JoseCT | stakely.io#8451
2021-08-26T13:04:38.754Z-YH | StakeWithUs#6077.slog.gz,YH | StakeWithUs#6077
2021-08-26T14:37:06.658Z-Calin | Stakeborg#1298.slog.gz,Calin | Stakeborg#1298
2021-08-26T14:49:46.108Z-Tibi K. | moonlet.io#6120.slog.gz,Tibi K. | moonlet.io#6120
2021-08-26T17:00:46.879Z-NAD010286#6785.slog.gz,NAD010286#6785
2021-08-26T19:12:30.842Z-mabalaru#6177.slog.gz,mabalaru#6177
2021-08-27T08:32:23.278Z-<-ThomaS->#0485.slog.gz,<-ThomaS->#0485
2021-08-27T08:34:10.061Z-Rok#8697.slog.gz,Rok#8697
2021-08-27T08:55:04.295Z-blackpaopao#7315.slog.gz,blackpaopao#7315


In [18]:
taskslog[taskslog['Discord ID'].str.contains('moon')]

,Discord ID,Moniker,Developer,Submission Link,Status,Verified,Last Date Updated
TaskBoardID,,,,,,,
4392,Tibi K. | moonlet.io #6120,Moonlet.io,Moonlet Wallet,storage key: 2021-08-26T15:01:39.785Z-Tibi K. ...,Completed,In review,2021-08-26 11:02:00


## Validators

## Blocks, Votes from the BigDipper Explorer DB

In [19]:
# !conda install --yes -c anaconda pymongo
# pymongo-3.11.0

# !pip install pymongo

# Successfully installed pymongo-3.12.0

See also:
 - [Connecting to MongoDB — Anaconda Platform 5\.5\.0 documentation](https://enterprise-docs.anaconda.com/en/latest/data-science-workflows/data/mongodb.html)
 - [collection – Collection level operations — PyMongo 3\.11\.4 documentation](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find)

_**Note:** we use an ssh tunnel to access the big dipper database._

In [20]:
def _connect(url='mongodb://localhost:27017/', name='stargate-agorictest-17'):
    import pymongo
    client = pymongo.MongoClient(url)
    return client.get_database(name)

_db = _connect()

The bigdipper refers to validators by address but we use moniker in the portal, so let's get the correspondence:

In [21]:
validator = pd.DataFrame.from_records(_db.validators.find())

validator['moniker'] = validator.description.apply(lambda d: d['moniker'])
validator = validator.set_index('address', drop=True)
validator[['moniker', 'operator_address', 'delegator_address']].head()

,moniker,operator_address,delegator_address
address,,,
E50B5E81A39DF687D3BFA36784B012A16A30CDF4,atlantean,agoricvaloper1qzttjkwdar7qxndh87aug8skl992dagn...,agoric1qzttjkwdar7qxndh87aug8skl992dagnh3x36l
3C2F54CE1E6D537275CF181CD363F8BFACC3240E,humantraffic,agoricvaloper1qytu7vsclm58mrk6vrg2nfcqcuxjwv43...,agoric1qytu7vsclm58mrk6vrg2nfcqcuxjwv43c4z2gq
0D8F5487E30DDED417D77ACBE102CBD92C67C781,ElectricCoinCo,agoricvaloper1q5mk8ap43g3s8pkxa0sx89s2zft5dxwg...,agoric1q5mk8ap43g3s8pkxa0sx89s2zft5dxwg7hpwqk
526504E4A42F6125762E443E59FA014788BA592C,stake2earn,agoricvaloper1qhmqp3weh5wx5da62mefc2mvku9p49um...,agoric1qhmqp3weh5wx5da62mefc2mvku9p49umpygznm
6A207A08A56A75B862D6CC3E4EA6937A17EA313A,Planemo,agoricvaloper1q6u4604ytuvgavvejrv9mw70y0gmcqex...,agoric1q6u4604ytuvgavvejrv9mw70y0gmcqexu4d690


### Info from review of submitted gentx files

See `gentx_clean.py` (_note: revision to include extracted moniker in the summary to appear_)

In [38]:
!ls -l portal-review/gentx17.csv

-rw-r--r-- 1 jupyter jupyter 11938 Aug 29 02:31 portal-review/gentx17.csv


In [22]:
gentx17 = pd.read_csv('portal-review/gentx17.csv', index_col=0, parse_dates=['Last Date Updated'])
# print(gentx17.dtypes)
gentx17.tail()

,TaskBoardID,Discord ID,Moniker,Status,jsonErr,Last Date Updated,moniker
145,3854,weilaikeqi#8418,kerwinnode,Completed,False,2021-08-19 13:18:00,kerwinnode
146,3715,windows | Nodeasy #4629,Nodeasy.com,Obsolete,True,2021-08-16 23:33:00,NaN
147,3716,windows | Nodeasy #4629,Nodeasy.com,Completed,False,2021-08-16 23:34:00,Nodeasy.com
148,3682,wiseoldman#8041,WM-Supernode,Completed,False,2021-08-16 13:17:00,WM-Supernode
149,3765,wombat#7690,wombat,Completed,False,2021-08-17 11:58:00,wombat


### Mismatch: monikers

Monker mismatches between knack portal records and submitted gentxs

In [23]:
gentx17ok = gentx17[(gentx17.Status == 'Completed') & (~gentx17.jsonErr)]
gentx17ok[gentx17ok.Moniker != gentx17ok.moniker]

,TaskBoardID,Discord ID,Moniker,Status,jsonErr,Last Date Updated,moniker
9,3791,Aliaksandr#3647,Coolcat,Completed,False,2021-08-17 13:50:00,CoolCat
19,3768,David Rakusan | Rockaway#1226,rbf.capital,Completed,False,2021-08-17 12:02:00,Rockaway Blockchain Fund
24,3747,Emre NOP#7479,EmreNOP,Completed,False,2021-08-17 08:42:00,EmreNOP | BlockSeal
37,3986,Jefferson Pham#2120,NaN,Completed,False,2021-08-23 07:42:00,STC Capital
59,3739,Restyle#3072,restyle,Completed,False,2021-08-17 07:14:00,neonodes
62,3772,Serg NI#8093,Niocrisnode,Completed,False,2021-08-17 12:16:00,niocrisN0de
67,3827,Shen | Bi23 Labs#6675,NaN,Completed,False,2021-08-19 05:34:00,Bi23 Labs
68,3766,Sjors#2650,Lemniscap,Completed,False,2021-08-17 11:58:00,NaN
79,3769,Víctor | melea#9821,melea ◮👁◭,Completed,False,2021-08-17 12:04:00,webex
87,3813,ahmedwk#6882,ahmed6174,Completed,False,2021-08-18 12:59:00,ahmed6175


## `gentx` Tasks

In [31]:
taskgen = tasksub[tasksub.Task == 'Create and submit gentx - Metering '][['TaskBoardID', 'Discord ID', 'Verified', 'Task', 'Status', 'Last Date Updated']]
taskgen.tail()

,TaskBoardID,Discord ID,Verified,Task,Status,Last Date Updated
4601,3789,Aliaksandr#3647,Not accepted,Create and submit gentx - Metering,Completed,2021-08-17 13:42:00
4602,3708,mihaildmm#2064,Not accepted,Create and submit gentx - Metering,Completed,2021-08-16 17:35:00
4603,3798,Trader_Payne#4602,Not accepted,Create and submit gentx - Metering,Completed,2021-08-17 15:03:00
4605,3793,Koen#4700,Not accepted,Create and submit gentx - Metering,Completed,2021-08-17 14:01:00
4617,3715,windows | Nodeasy #4629,Not accepted,Create and submit gentx - Metering,Completed,2021-08-16 23:33:00


In [32]:
taskgen[~taskgen.TaskBoardID.isin(gentx17.TaskBoardID)]

,TaskBoardID,Discord ID,Verified,Task,Status,Last Date Updated


In [60]:
# pd.merge(taskgen, gentx17, on='Discord ID', how='outer')

In [34]:
taskgen[~taskgen.TaskBoardID.isin(gentx17.TaskBoardID)]

,TaskBoardID,Discord ID,Verified,Task,Status,Last Date Updated


## Matching `gentx`s Validators

matching `gentx` task submissions with validators from the Explorer

In [35]:
genval = pd.merge(gentx17, validator, on='moniker', how='outer')
# print(x.dtypes)
genval[~genval.status.isnull() & ~genval.TaskBoardID.isnull()][['TaskBoardID', 'Discord ID', 'moniker', 'accpub', 'delegator_address', 'status', 'tokens']]

,TaskBoardID,Discord ID,moniker,accpub,delegator_address,status,tokens
0,3732.0,#0485,AlphaBravo,agoricpub1zcjduepq9ca6vajqwryy9h5hn6k99pwu6mz6...,agoric1u66kx826y2x87u7url0jzsaludvk6dxn473mxs,BOND_STATUS_BONDED,50000000.0
2,3780.0,13610152128#5923,nodestradamus,agoricpub1zcjduepqwp48d495hw4c0n24ud047sxshh07...,agoric18tmvfjlefz0tz3pumypa99q7sqcpzwqd08c8gy,BOND_STATUS_UNBONDING,47560000.0
3,3785.0,86b#6313,oraclegen,agoricpub1zcjduepqqrrk6qjl3x45pe4qw0yua06kuy0z...,agoric1k9umcy8tkjq7yhavyude477h2uadvtjuxn30mg,BOND_STATUS_UNBONDING,47560000.0
4,3723.0,@calin_stakeborg,Stakeborg,agoricpub1zcjduepq7w9x9w5en7kr2qnfawucplcdvsjy...,agoric1s4tqjr9ln6kcqaq9ljt26xr504z8lfkglaelqy,BOND_STATUS_UNBONDING,49010000.0
5,3680.0,Adorid | adorid.xyz#1293,Adorid,agoricpub1zcjduepqaqnrwtztc3e7ygl28sqmelajyhml...,agoric1pfdehd8tkx49wx6lf4t2en4ts4srkmf6fw30kz,BOND_STATUS_UNBONDING,49500000.0
...,...,...,...,...,...,...,...
144,3853.0,weilaikeqi#8418,kerwinnode,agoricpub1zcjduepqv2c9krhtfqn8z7x970k3muum6az7...,agoric1fp0ly25557k5mj5hepcc3vrwva2metpt6jlewf,BOND_STATUS_UNBONDING,49010000.0
145,3855.0,weilaikeqi#8418,kerwinnode,agoricpub1zcjduepqv2c9krhtfqn8z7x970k3muum6az7...,agoric1fp0ly25557k5mj5hepcc3vrwva2metpt6jlewf,BOND_STATUS_UNBONDING,49010000.0
146,3854.0,weilaikeqi#8418,kerwinnode,agoricpub1zcjduepqv2c9krhtfqn8z7x970k3muum6az7...,agoric1fp0ly25557k5mj5hepcc3vrwva2metpt6jlewf,BOND_STATUS_UNBONDING,49010000.0
147,3716.0,windows | Nodeasy #4629,Nodeasy.com,agoricpub1zcjduepqp3tr82jz4ux2vzddlrc8x2kyrjkd...,agoric13de33dfcz7qcmmvfmrj72mhn6df3s36nywawn2,BOND_STATUS_UNBONDING,49010000.0


### Mismatch: gentx tasks

Submissions with no matching validator from the explorer:

In [36]:
genval[genval.status.isnull()][['TaskBoardID', 'Discord ID', 'moniker', 'accpub', 'delegator_address', 'status', 'tokens']]

,TaskBoardID,Discord ID,moniker,accpub,delegator_address,status,tokens
1,3797.0,0xd18b,0xd18b,NaN,NaN,NaN,NaN
13,3842.0,CRT.Vinnylee (Will Not DM You)#6565,crt,NaN,NaN,NaN,NaN
29,3836.0,GabbyYue#6705,gab,NaN,NaN,NaN,NaN
37,3986.0,Jefferson Pham#2120,STC Capital,NaN,NaN,NaN,NaN
42,3793.0,Koen#4700,NaN,NaN,NaN,NaN,NaN
43,3804.0,Shai#1550,NaN,NaN,NaN,NaN,NaN
44,3766.0,Sjors#2650,NaN,NaN,NaN,NaN,NaN
45,3801.0,Trader_Payne#4602,NaN,NaN,NaN,NaN,NaN
46,3702.0,mihaildmm#2064,NaN,NaN,NaN,NaN,NaN
47,3715.0,windows | Nodeasy #4629,NaN,NaN,NaN,NaN,NaN


### Mismatch: Validators

with no matching Task Submission

In [37]:
genval[genval.TaskBoardID.isnull()][['TaskBoardID', 'Discord ID', 'moniker', 'accpub', 'delegator_address', 'status', 'tokens']]

,TaskBoardID,Discord ID,moniker,accpub,delegator_address,status,tokens
150,NaN,NaN,Lemniscap,agoricpub1zcjduepqyh44t62e3h03gvjzvehwz7rdxca0...,agoric1ch738c0t2xkykz495wmszfl4gql79e4zyp9u4t,BOND_STATUS_UNBONDING,49500000.0
151,NaN,NaN,Stake.su,agoricpub1zcjduepq7wz0plddrt49gx8j3aaltuec8r46...,agoric19v6m462ca5kz73gz9l9ylpdag8hpuzv3tt6prd,BOND_STATUS_BONDED,48040000.0
152,NaN,NaN,4andr,agoricpub1zcjduepqqn7mf4frf4grkzvrufyfl68008gg...,agoric1757ehueaqzl8gm822c6pq3lpf3atvsrtx5vr90,BOND_STATUS_UNBONDING,47560000.0


## A Canonical Slogfile

In [58]:
sf0 = sf5[sf5['size'] == sf5['size'].max()].rename(columns=dict(discordID='Discord ID'))
sf0.drop(columns=['path'])

,size,modified,fresh,Discord ID
filename,,,,
2021-08-27T11:58:28.246Z-Solysium#9645.slog.gz,14321737146,2021-08-27 12:20:16+00:00,2021-08-27 11:58:28.246,Solysium#9645


In [59]:
pd.merge(sf0.reset_index(), genval[['Discord ID', 'moniker', 'delegator_address']]).drop(columns=['path'])

,filename,size,modified,fresh,Discord ID,moniker,delegator_address
0,2021-08-27T11:58:28.246Z-Solysium#9645.slog.gz,14321737146,2021-08-27 12:20:16+00:00,2021-08-27 11:58:28.246,Solysium#9645,atlantean,agoric1qzttjkwdar7qxndh87aug8skl992dagnh3x36l
